In [4]:
import pandas as pd
import numpy as np
from datetime import date
import cpi
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
pd.set_option('display.max_columns', 500)
main_df = pd.read_csv(r'/home/shua/Desktop/Housetable/Recommender/price_prediction/Pricing_Model_dev/BATCH_API/data/MLS_Reggie/mls_tagged_with API.csv').drop(['Unnamed: 0'], axis = 1)

/tmp/ipykernel_40555/3480252297.py:9: DtypeWarning: Columns (6,9,83,140,305,306,307,308,317,333,337,338,339,345,346,348,350,351,352,354,355,357,358,359,360,365) have mixed types. Specify dtype option on import or set low_memory=False.
  main_df = pd.read_csv(r'/home/shua/Desktop/Housetable/Recommender/price_prediction/Pricing_Model_dev/BATCH_API/data/MLS_Reggie/mls_tagged_with API.csv').drop(['Unnamed: 0'], axis = 1)


In [5]:
main_df

,full_address,address,street,city,state,zip,sold_month,sold_day,year_sold,_id,homeownerAssociations,address.oldHashes,address.houseNumber,address.county,address.zipPlus4,address.localities,address.hash,address.latitude,address.longitude,address.normalized,address.countyFipsCode,address.formattedStreet,address.streetNoUnit,address.geoStatusCode,address.geoStatus,assessment.assessedImprovementValue,assessment.assessedLandValue,assessment.totalAssessedValue,assessment.assessmentYear,building.totalBuildingAreaSquareFeet,building.totalBuildingAreaCode,building.totalBuildingAreaCodeDescription,building.yearBuilt,building.effectiveYearBuilt,building.buildingCount,building.roomCount,building.unitCount,building.bedroomCount,building.calculatedBathroomCount,building.fullBathroomCount,building.bathroomCount,building.storyCode,building.storyCount,building.storyCountDescription,building.garageParkingSpaceCount,building.residentialUnitCount,building.features,building.airConditioningSourceCode,building.airConditioningSource,building.buildingConditionCode,building.buildingCondition,building.buildingQualityCode,building.buildingQuality,building.constructionTypeCode,building.constructionType,building.exteriorWallsCode,building.exteriorWalls,building.heatSourceCode,building.heatSource,building.roofCoverCode,building.roofCover,building.roofTypeCode,building.roofType,building.styleCode,building.style,demographics.age,demographics.householdSize,demographics.income,demographics.netWorth,demographics.discretionaryIncome,demographics.genderCode,demographics.gender,demographics.millionaire,demographics.religiousAffiliationCode,demographics.religiousAffiliation,demographics.individualEducationCode,demographics.individualEducation,demographics.recentlyMoved,demographics.recentlyMovedMonth,demographics.recentlyMovedYear,general.primaryParcel,general.parcelCount,general.vacant,general.mailingAddressVacant,general.standardizedLandUseCode,general.propertyTypeCategory,general.propertyTypeDetail,general.congressionalDistrict,general.timeZone,general.utcOffset,general.daylightSavingsTime,general.carrierRoute,general.censusTract,general.schoolDistrict,ids.addressHash,ids.apn,ids.quantariumUrn,ids.fipsCode,ids.personIds,involuntaryLien.liens,legal.legalDescription,legal.lotNumber,legal.cityTownshipMunicipality,legal.tractNumber,lot.lotSizeAcres,lot.lotDepthFeet,lot.lotFrontageFeet,lot.lotSizeSquareFeet,lot.zoningCode,lot.estimatedLotSizeSquareFeet,mls.mlsNumber,mls.status,mls.price,mls.daysOnMarket,mls.originalListingDate,mls.initialListingStatus,mls.statusSubtype,mls.rentalIndicator,mls.maxListPrice,mls.maxListPriceDate,mls.minListPrice,mls.minListPriceDate,mls.soldPrice,mls.soldDate,mls.bedroomCount,mls.bathroomCount,mls.buildingStyle,mls.totalBuildingAreaSquareFeet,mls.lotSizeSquareFeet,mls.floorCount,mls.propertyType,mls.appliances,mls.coolingTypes,mls.heatingTypes,mls.agentOffices,mls.schools,mls.agents,openLien.mortgages,owner.fullName,owner.mailingAddress.houseNumber,owner.mailingAddress.street,owner.mailingAddress.city,owner.mailingAddress.county,owner.mailingAddress.state,owner.mailingAddress.zip,owner.mailingAddress.formattedStreet,owner.mailingAddress.streetNoUnit,owner.mailingAddress.hash,owner.names,owner.ownerOccupied,owner.ownerStatusType,owner.lengthOfResidenceMonths,owner.ownershipStartDate,quickLists.absenteeOwner,quickLists.absenteeOwnerInState,quickLists.absenteeOwnerOutOfState,quickLists.activeListing,quickLists.activeAuction,quickLists.bankruptcy,quickLists.cashBuyer,quickLists.corporateOwned,quickLists.divorce,quickLists.expiredListing,quickLists.freeAndClear,quickLists.highEquity,quickLists.inherited,quickLists.liens,quickLists.listedBelowMarketPrice,quickLists.lowEquity,quickLists.mailingAddressVacant,quickLists.noticeOfDefault,quickLists.onMarket,quickLists.outOfStateOwner,quickLists.ownerOccupied,quickLists.pendingListing,quickLists.preforeclosure,quickLists.recentlySold,quickLists.samePropertyAndMailingAddress,quickLists.taxDefault,qui

In [6]:
df_dropped_na = main_df.dropna(subset = ['price_sold_tagged', 'full_address', 'year_sold', 'sold_month', 'sold_day', 'address', 'address.houseNumber', 'street', 'city',
        'state', 'zip', 'Bathroom_max_tagged',
 'Bathroom_min_tagged',
 'Bathroom_std_tagged',
 'Bathroom_mean_tagged',
 'Floor_max_tagged',
 'Floor_min_tagged',
 'Floor_std_tagged',
 'Floor_mean_tagged',
 'Walls_max_tagged',
 'Walls_min_tagged',
 'Walls_std_tagged',
 'Walls_mean_tagged',
 'Outview_tagged',
 'Outview_std_tagged'], axis = 0)
df_dropped_na.shape

(4632, 385)

In [7]:
from re import sub
from decimal import Decimal
def convert_money(x):
    try:
        return float(x)
    except:
        return Decimal(sub(r'[^\d.]', '', x))
df_dropped_na.price_sold_tagged = df_dropped_na.price_sold_tagged.apply(convert_money)

/tmp/ipykernel_40555/3447556188.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_dropped_na.price_sold_tagged = df_dropped_na.price_sold_tagged.apply(convert_money)


In [8]:
dub_addresses = df_dropped_na[df_dropped_na[['street']].duplicated(['street'])]['street']
dub_df = df_dropped_na.loc[df_dropped_na['street'].isin(dub_addresses)]
dub_df.shape

(4325, 385)

In [26]:
dub_addresses

3           3913 york blvd
5         4171 division st
7          4234 palmero dr
10      4237 caledonia way
12         4613 jessica dr
               ...        
4726       1814 e 109th st
4729       2217 e 117th st
4731    2013 n corlett ave
4733       2307 e 120th st
4737     11709 compton ave
Name: street, Length: 2203, dtype: object

In [9]:
dub_df.groupby(['street'])[['Bathroom_max_tagged',
 'Bathroom_min_tagged',
 'Bathroom_std_tagged',
 'Bathroom_mean_tagged',
 'Floor_max_tagged',
 'Floor_min_tagged',
 'Floor_std_tagged',
 'Floor_mean_tagged',
 'Walls_max_tagged',
 'Walls_min_tagged',
 'Walls_std_tagged',
 'Walls_mean_tagged',
 'Outview_tagged',
 'Outview_std_tagged']].std()
columns = ['full_address', 'address', 'sale_between', 'Bathroom_max_tagged',
 'Bathroom_min_tagged',
 'Bathroom_std_tagged',
 'Bathroom_mean_tagged',
 'Floor_max_tagged',
 'Floor_min_tagged',
 'Floor_std_tagged',
 'Floor_mean_tagged',
 'Walls_max_tagged',
 'Walls_min_tagged',
 'Walls_std_tagged',
 'Walls_mean_tagged',
 'Outview_tagged',
 'Outview_std_tagged']

aggregation_columns = ['Bathroom_max_tagged',
 'Bathroom_min_tagged',
 'Bathroom_std_tagged',
 'Bathroom_mean_tagged',
 'Floor_max_tagged',
 'Floor_min_tagged',
 'Floor_std_tagged',
 'Floor_mean_tagged',
 'Walls_max_tagged',
 'Walls_min_tagged',
 'Walls_std_tagged',
 'Walls_mean_tagged',
 'Outview_tagged',
 'Outview_std_tagged']
 
difference_df = pd.DataFrame(columns = columns)

In [38]:
list_of_data = []
count =0
for house in dub_df['street'].unique():
    house_df = dub_df[dub_df['street'] == house].sort_values(by = ['year_sold', 'sold_month'], ascending = [False, False]).reset_index(drop = True)
    #order the sales by their sale date
    if house_df.shape[0] >2:
        count+=1
    for i, sale_a in house_df.iterrows():
        # print(i)
        # print(sale_a['year_sold'])
        # sale_a = house_df.iloc[i]
        sale_a = house_df.iloc[i]
        try:

            sale_b = house_df.iloc[i+1]
    #         # print(i)
    #         # print(sale_a['street'])
    #         for j, sale_b in house_df.iterrows():
    #             if sale_a > sale_b (don't want the model to predict against itself. Or do we?)
    #             #     continue
            data_point = dict()
            data_point['full_address'] = sale_a['full_address']
            data_point['street'] = house
            data_point['sale_year'] = sale_a['year_sold']
            data_point['prev_sale_year'] = sale_b['year_sold']
            # data_point['time_between_sales'] = int(sale_a['year_sold']) - int(sale_b['year_sold']) #Need to fix in pre-processing

            # print(data_point['sale_between'])
            data_point['sale_price'] = sale_a['price_sold_tagged']
            data_point['prev_sale_price'] = sale_b['price_sold_tagged']
            data_point['price_difference'] = int(float(sale_a['price_sold_tagged'])) - int(float(sale_b['price_sold_tagged']))
            # print(data_point['price_difference'])
            # for col in aggregation_columns:
            #     data_point[col] = int(sale_a[col]) - int(sale_b[col])
            list_of_data.append(data_point)
        except:
            data_point = dict()
            data_point['full_address'] = sale_a['full_address']
            data_point['street'] = house
            data_point['sale_year'] = sale_a['year_sold']
            data_point['prev_sale_year'] = None
            # data_point['time_between_sales'] = int(sale_a['year_sold']) - int(sale_b['year_sold']) #Need to fix in pre-processing

            # print(data_point['sale_between'])
            data_point['sale_price'] = sale_a['price_sold_tagged']
            data_point['prev_sale_price'] = None
            data_point['price_difference'] = None
            # print(data_point['price_difference'])
            # for col in aggregation_columns:
            #     data_point[col] = None
            list_of_data.append(data_point)

print(count)    
difference_df = pd.DataFrame.from_records(list_of_data)


76


In [39]:
difference_df

,full_address,street,sale_year,prev_sale_year,sale_price,prev_sale_price,price_difference
0,"3913 York Boulevard, Los Angeles, CA 90065 Sol...",3913 york blvd,2021,2019,1030000.0,725000.0,305000.0
1,"3913 York Boulevard, Los Angeles, CA 90065 Sol...",3913 york blvd,2019,None,725000.0,NaN,NaN
2,"4171 Division Street, Los Angeles, CA 90065 So...",4171 division st,2020,2015,905000.0,680000.0,225000.0
3,"4171 Division Street, Los Angeles, CA 90065 So...",4171 division st,2015,None,680000.0,NaN,NaN
4,"4234 Palmero Drive, Los Angeles, CA 90065 Sold...",4234 palmero dr,2020,2017,1260000.0,844000.0,416000.0
...,...,...,...,...,...,...,...
4320,"2013 N Corlett Avenue, Los Angeles, CA 90059 S...",2013 n corlett ave,2016,None,200000.0,NaN,NaN
4321,"2307 E 120th St, Los Angeles, CA 90059 Sold Au...",2307 e 120th st,2021,2016,560000.0,335000.0,225000.0
4322,"2307 E 120th Street, Los Angeles, CA 90059 Sol...",2307 e 120th st,2016,None,335000.0,NaN,NaN
4323,"11709 Compton Avenue, Los Angeles, CA 90059 So...",11709 compton ave,2017,2017,200000.0,255000.0,-55000.0


In [40]:
difference_df = difference_df.merge(dub_df, how = 'left', on = 'full_address', suffixes= ('', '_b'))

In [43]:
difference_df.columns

Index(['full_address', 'street', 'sale_year', 'prev_sale_year', 'sale_price',
       'prev_sale_price', 'price_difference', 'address', 'street_b', 'city',
       ...
       'Floor_min_tagged', 'Floor_std_tagged', 'Floor_mean_tagged',
       'Walls_max_tagged', 'Walls_min_tagged', 'Walls_std_tagged',
       'Walls_mean_tagged', 'Outview_tagged', 'Outview_std_tagged',
       's_tagged'],
      dtype='object', length=391)

In [42]:
difference_df.to_csv('/home/shua/Desktop/msc-ml-datamining/Machine_Learning/project/difference_df.csv')